In [12]:
import pandas as pd
data = pd.read_csv('preprocessed_data_with_numerical.csv')

In [13]:
data.fillna('', inplace=True)

In [14]:
data['full_text'] = data['title'] + " " + data['location']  + " " + data['department']  + " " + data['company_profile']  + " " + data['description']  + " " + data['requirements']  + " "  + data['benefits'] + data['industry']  + " " + data['function']

In [15]:
data

,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset,missing company profile,missing company information,full_text
0,marketing intern,usa ny new york,marketing,food52 created groundbreaking award winning co...,food52 fast growing james beard award winning ...,experience content management system major plu...,,0,1,0,2,4,9,,marketing,0,0,1,3,marketing intern usa ny new york marketing foo...
1,customer service cloud video production,nz auckland,success,90 second world cloud video production service...,organised focused vibrant awesome passion cust...,expect key responsibility communicate client 9...,get u part 90 second team gain experience work...,0,1,0,1,6,9,marketing advertising,customer service,0,0,1,3,customer service cloud video production nz auc...
2,commissioning machinery assistant cma,usa ia wever,,valor service provides workforce solution meet...,client located houston actively seeking experi...,implement pre commissioning commissioning proc...,,0,1,0,2,6,9,,,0,0,1,3,commissioning machinery assistant cma usa ia w...
3,account executive washington dc,usa dc washington,sale,passion improving quality life geography heart...,company esri environmental system research ins...,education bachelor master gi business administ...,culture anything corporate collaborative creat...,0,1,0,1,5,1,computer software,sale,0,0,1,3,account executive washington dc usa dc washing...
4,bill review manager,usa fl fort worth,,spotsource solution llc global human capital m...,job title itemization review manager location ...,qualification rn license state texas diploma b...,full benefit offered,0,1,1,1,5,1,hospital health care,health care provider,0,0,1,3,bill review manager usa fl fort worth spotsou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,account director distribution,ca toronto,sale,vend looking awesome new talent come join u wo...,case first time visited website vend award win...,ace role eat comprehensive statement work brea...,expect u open culture openly share result inpu...,0,1,1,1,5,9,computer software,sale,0,0,1,3,account director distribution ca toronto sale ...
17876,payroll accountant,usa pa philadelphia,accounting,weblinc e commerce platform service provider f...,payroll accountant focus primarily payroll fun...,b b accounting desire fun love genuine passion...,health wellness medical plan prescription drug...,0,1,1,1,5,1,internet,accounting auditing,0,0,1,3,payroll accountant usa pa philadelphia account...
17877,project cost control staff engineer cost contr...,usa tx houston,,provide full time permanent position many medi...,experienced project cost control staff enginee...,least 12 year professional experience ability ...,,0,0,0,1,6,9,,,0,0,1,2,project cost control staff engineer cost contr...
17878,graphic designer,ng la lagos,,,nemsia studio looking experienced visual graph...,1 must fluent latest version corel adobe cc es...,competitive salary compensation based experien...,0,0,1,0,6,6,graphic design,design,0,0,0,0,graphic designer ng la lagos nemsia studio l...


In [31]:
X = data[['full_text', 'has_questions', 'required_experience', 'required_education', 'missing company information']]
# X = data['full_text']
y = data['fraudulent']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
# from sklearn.pipeline import FeatureUnion, Pipeline
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

<h1>Text Only Models</h1>

TFIDF Vectorizer

In [33]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['full_text'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['full_text'])

In [34]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Step 4: Logistic Regression
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

Logistic Regression F1 Score: 0.625
SVM F1 Score: 0.8153846153846154
Random Forest F1 Score: 0.7874015748031495
XGBoost F1 Score: 0.8226600985221675


Word2Vec

In [25]:
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

# Step 2: Tokenize the text
tokenized_text = X_train['full_text'].apply(lambda x: x.split())

# Step 3: Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Step 4: Convert words to vectors
def get_vector(word_list, model):
    valid_words = [word for word in word_list if word in model.wv]
    if not valid_words:
        # If no valid words, return a vector of zeros or handle as needed
        return np.zeros(model.vector_size)
    return np.mean([model.wv[word] for word in valid_words], axis=0)

X_train_word2vec = tokenized_text.apply(lambda x: get_vector(x, word2vec_model))
X_test_word2vec = X_test['full_text'].apply(lambda x: get_vector(x.split(), word2vec_model))

In [29]:
# Step 5: Train models and evaluate F1 score
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train.to_list(), y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test.to_list())
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Step 6: Logistic Regression
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 7: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 8: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

/Users/andreayeo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression F1 Score: 0.4011799410029498
SVM F1 Score: 0.5093167701863354
Random Forest F1 Score: 0.5714285714285714
XGBoost F1 Score: 0.6989247311827957


CountVectorizer

In [23]:
count_vectorizer = CountVectorizer()
X_train_cv = count_vectorizer.fit_transform(X_train['full_text'])
X_test_cv = count_vectorizer.transform(X_test['full_text'])

In [24]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Step 4: Logistic Regression
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

/Users/andreayeo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression F1 Score: 0.8009153318077803
SVM F1 Score: 0.7272727272727272
Random Forest F1 Score: 0.793733681462141
XGBoost F1 Score: 0.8382352941176471


<h3>Using text features only</h3>

|  | Logistic Regression | SVM | Random Forest | XGBoost | Average |
|----------|----------|----------|----------|----------|----------|
| TFIDF | 0.625 | 0.815 | 0.787 | 0.827 | 0.764 |
| Word2Vec | 0.401 | 0.509 | 0.571 | 0.699 | 0.545 |
| CountVectorizer | 0.801 | 0.727 | 0.794 | 0.838 | 0.790 |


Based on the results above, the best word embedding method to use is CountVectorizer that obtained the highest average F1 score of 0.790 across all models. Hence, we will be using CountVectorizer moving forward.

<h1>Combined text and numeric</h1>

tfidf and numeric

In [53]:
numeric_features = X_train[['has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education']]
combined_features = np.hstack([
    StandardScaler().fit_transform(numeric_features),
    X_train_tfidf.toarray()])
X_test_numeric_features = X_test[['has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education']]
X_test_combined_features = np.hstack([
    StandardScaler().fit_transform(X_test_numeric_features),
    X_test_tfidf.toarray()])

def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

/Users/andreayeo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression F1 Score: 0.5952380952380952
SVM F1 Score: 0.6705539358600584
Random Forest F1 Score: 0.80719794344473


In [51]:
train_word2vec_array = np.vstack(X_train_word2vec.apply(lambda x: x.flatten()).to_numpy())
test_word2vec_array = np.vstack(X_test_word2vec.apply(lambda x: x.flatten()).to_numpy())
combined_features = np.hstack([
    StandardScaler().fit_transform(numeric_features),
    train_word2vec_array])
X_test_numeric_features = X_test[['has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education']]
X_test_combined_features = np.hstack([
    StandardScaler().fit_transform(X_test_numeric_features),
    test_word2vec_array])

def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

/Users/andreayeo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression F1 Score: 0.4057142857142857
SVM F1 Score: 0.5504587155963303
Random Forest F1 Score: 0.5945945945945946


In [35]:
numeric_features = X_train[['has_questions', 'required_experience', 'required_education', 'missing company information']]
combined_features = np.hstack([
    StandardScaler().fit_transform(numeric_features),
    X_train_cv.toarray()])
X_test_numeric_features = X_test[['has_questions', 'required_experience', 'required_education', 'missing company information']]
X_test_combined_features = np.hstack([
    StandardScaler().fit_transform(X_test_numeric_features),
    X_test_cv.toarray()])

def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

/Users/andreayeo/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression F1 Score: 0.8232662192393735
SVM F1 Score: 0.7287671232876712
Random Forest F1 Score: 0.7905759162303664
XGBoost F1 Score: 0.8463356973995273


<h3>Combined features</h3>

|  | Logistic Regression | SVM | Random Forest | XGBoost |
|----------|----------|----------|----------|----------|
| Text Only | 0.801 | 0.727 | 0.794 | 0.838 |
| Text and Numeric | 0.823 | 0.729 | 0.791 | 0.846 |

In [ ]:
# neural network

import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import f1_score

# Define and compile the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model on the training data
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Make predictions on the validation data
y_pred_valid = model.predict(X_val)
y_pred_valid = (y_pred_valid > 0.5).astype(int)  # Convert probabilities to binary labels

# Calculate the F1 score on the validation set
f1_valid = f1_score(y_val, y_pred_valid)

print("Validation Set F1 Score:")
print(f"F1 Score: {f1_valid}")

# Make predictions on the test data
y_pred_test = model.predict(X_test)
y_pred_test = (y_pred_test > 0.5).astype(int)  # Convert probabilities to binary labels

# Calculate the F1 score on the test set
f1_test = f1_score(y_test, y_pred_test)

print("Test Set F1 Score:")
print(f"F1 Score: {f1_test}")